In [ ]:
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import seaborn as sns

In [ ]:
sns.set()

# Exercise session 3: Sampling with Metropolis and MCMC

We stick with the good old Beta-Binomial problem:

Let us consider a coin that lands heads with probability $\theta=0.25$ (assumed unknown). 

Our prior knowledge of $\theta$ is encoded in a Beta distributions with parameters $a=20$ and $b=10$:

$$ f_{\rm prior}(\theta) = \frac{1}{B(a,b)} \theta^{a-1} (1-\theta)^{b-1}, \qquad \theta \in (0, 1),$$

where $B(a,b)$ is the normalization constant such that $\int_{0}^1 f_{\rm prior}(\theta) \; d \theta = 1.$ 

We toss the coin $n$ times and measure a number $y$ of heads. How does our belief of $\theta$ change with the measurement?

In [ ]:
a = 20 # prior: parameter a
b = 10 # prior: parameter b
n = 50 # likelihood: number of tosses
y = 15 # likelihood: number of HEADs observed. (Fixed in this example. It could be sampled from a binomial instead)
#np.random.seed(42)
#y = np.random.binomial(n, theta) # random number of heads from the distribution from a binomial distribution

## Approach 1: Exact derivation

* Obtain the posterior distribution $f_{\rm post}(\theta) = f(\theta | y)$ in closed-form:

$$f_{\rm post}(\theta) = \frac{1}{B(a+y,b+n-y)} \theta^{a+y-1} (1-\theta)^{b+n-y-1}, \qquad \theta \in (0, 1)$$

* Define random variables corresponding to the prior and the posterior in Python using `scipy.stats.beta`

In [ ]:
a_post = a + y
b_post = b + n - y
prior_var = stats.beta(a=a, b=b)
post_var = stats.beta(a=a_post, b=b_post) # conjugate prior-likelihood case, closed-form expression known

* Plot the prior and posterior distribution of $\theta$, together with 95% HDI

In [ ]:
dtheta = 1e-3 # discretization step for theta
theta_vec = np.arange(0, 1, dtheta) # discretized theta range
alpha = 0.05
plt.plot(theta_vec, prior_var.pdf(theta_vec), label="prior", color="C0")
prior_L, prior_H = prior_var.interval(1-alpha)
plt.axvspan(xmin=prior_L, xmax=prior_H, alpha=0.1, color="C0")
plt.plot(theta_vec, post_var.pdf(theta_vec), label="posterior", color="C1")
post_L, post_H = post_var.interval(1-alpha)
plt.axvspan(xmin=post_L, xmax=post_H, alpha=0.1, color="C1")
#plt.axvline(theta_0, color="black", label=r"true $\theta$")
plt.xlabel(r"$\theta$")
plt.legend()
plt.title(r"Prior and posterior distribution of $\theta$");

* What are the limitations of the exact derivation approach?

It is only applicable to very special cases (conjugate prior/likelihood pairs).

## Approach 2: Gridding

* Implement the prior, the likelihood, and the posterior (all up to a scaling factor independent of $\theta$) as Python functions

\begin{align}
\text{prior}: f_{\rm prior}(\theta) &\propto  \theta^{a-1} (1-\theta)^{b-1}\\
\text{likelihood}: \mathcal{L}(\theta) &\propto P(y|\theta) = {{n}\choose{y}} \theta^{y} \cdot (1-\theta)^{n-y},\\
\text{posterior}: f_{\rm post}(\theta) &\propto f_{\rm prior}(\theta) \mathcal{L}(\theta)
\end{align}


In [ ]:
prior_fun = lambda theta: stats.beta.pdf(theta, a, b)
lik_fun = lambda theta: (theta ** y) * (1 - theta)**(n-y)

def unnorm_prior_fun(theta):
    p_theta = theta**(a-1)*(1-theta)**(b-1) # since we look for an unnormalized posterior, this is also good enough
    p_theta = p_theta * ((theta <= 1.0) & (theta >= 0.0))
    return p_theta
    
unnorm_post_fun = lambda theta: unnorm_prior_fun(theta)*lik_fun(theta)

* Plot the unnormalized posterior

In [ ]:
dtheta = 1e-3
theta_vec = np.arange(0, 1, dtheta)
plt.plot(theta_vec, unnorm_post_fun(theta_vec))
plt.xlabel(r"$\theta$");
plt.title(r"Unnormalized posterior distribution of $\theta$");

* Obtain the normalized posterior by normalizing the unnormalized posterior

In [ ]:
dtheta = 1e-3
theta_vec = np.arange(0, 1, dtheta)
Z = np.sum(unnorm_post_fun(theta_vec)) * dtheta
normalized_post_fun = lambda theta: unnorm_post_fun(theta)/Z

* Verify that the gridding approximation of the posterior is close to the exact solution

In [ ]:
plt.plot(theta_vec, post_var.pdf(theta_vec), label="Exact")
plt.plot(theta_vec, normalized_post_fun(theta_vec), label="Normalized")
plt.xlabel(r"$\theta$");
plt.legend()
plt.title(r"Posterior distribution of $\theta$");

* What are the limitations of the gridding approach?

It is only applicable in 1,2,3 dimensions as it suffers the curse of dimensionality. 

NOTE: It's better to implement the gridding approach in the log-domain (as done in the previous lab session)

## Approach 3: Metropolis

Let us now pretend that we can only evaluate only evaluate $f_{\rm post}(\theta)$ up to an unknown multiplicative factor, which is in general the case for non-conjugate prior/likelihood pairs, where all we can say is that 
$f_{\rm post}(\theta) \propto f_{\rm prior}(\theta) \mathcal{L}(\theta)$.
* Implement a Metropolis algorithm to obtain a sample-based approximation of the posterior $f_{\rm post}(\theta)$. 
    * Use as target distribution the unnormalized posterior `unnorm_post_fun`
    * Use the following proposal:
    $$\theta_{\rm proposal} = \theta_{\rm current} + \Delta \theta, \qquad \Delta \theta \sim \mathcal{N}(0, \sigma^2).$$
    * Set $\sigma = 0.1$ and initialize $\theta_{\rm current} = 0.5$
    * Run a single chain of length 50000.
    
Hint: applicable MCMC algorithm  is described at pages 17-18 of lecture notes "Markov Chain Monte Carlo"


In [ ]:
sigma = 0.1 # proposal standard deviation
theta_0 = 0.5 # an initial guess
N = 50_000 # chain length

In [ ]:
theta_step = theta_0
p_step = unnorm_post_fun(theta_step)
thetas = [] # here we collect the theta values from the chain

for idx in range(N):
    thetas.append(theta_step)
    
    theta_prop = theta_step + sigma*np.random.randn()
    theta_prop = np.clip(theta_prop, 0, 1)
    p_prop = unnorm_post_fun(theta_prop) # evaluate unscaled distribution
  
    p_ratio = p_prop/p_step # the multiplicative factor disappears from the ratio!
    if p_ratio > 1:
        accept = True
    else:
        accept =  np.random.binomial(1, p_ratio) # othewise, accept w.p. p_ratio
        #accept = scipy.stats.bernoulli(p_ratio).rvs() # way slower!
    
    if accept:
        theta_step = theta_prop
        p_step = p_prop

In [ ]:
#thetas = thetas[1000:]

* Compare the Metropolis samples with the theoretical distribution. What do you observe?

In [ ]:
plt.hist(thetas, density=True, bins=50, label="Metropolis samples")
plt.plot(theta_vec, post_var.pdf(theta_vec), label="Exact posterior")
plt.legend()
plt.xlim([0, 1]);

The MCMC samples seem to approximate the true posterior very well!

## Logarithm implementation 
The unnormalized posterior distribution has a bad numerical scaling (in this case, and in general!). In some cases, this can become an issue.

The standard approach is to work out all operations in log-domain:

\begin{align*}
\ell(\theta) &= \log \mathcal{L}(\theta) = {n \choose y} + y \log (\theta) + (n-y) \log (1-\theta) \\
 \log f_{\rm prior}(\theta) &= -\log({B(a,b)}) + (a-1) \log (\theta) + (b-1)\log(1-\theta) \\
 \log f_{\rm post}(\theta) &= \ell(\theta) + \log f_{\rm prior}(\theta) + \log f(y).
\end{align*}


In [ ]:
def log_lik_fun(theta):

    eps = 1e-12
    theta = np.minimum(theta, 1 - eps)
    theta = np.maximum(theta, eps)
    
    log_lik =  y * np.log(theta) + (n-y)*np.log(1 - theta)
    return log_lik

def unnorm_log_prior_fun(theta): # otherwise, use prior_dist.logpdf
    
    eps = 1e-12
    theta = np.minimum(theta, 1 - eps)
    theta = np.maximum(theta, eps)

    log_f_theta = (a-1)*np.log(theta) + (b-1)*np.log(1-theta) 
    #log_f_theta = log_f_theta * ((theta <= 1.0) & (theta >= 0.0)) # 
    return log_f_theta

unnorm_log_post_fun = lambda theta: log_lik_fun(theta) + unnorm_log_prior_fun(theta)

In [ ]:
dtheta = 1e-3
theta_vec = np.arange(0, 1, dtheta)
plt.plot(theta_vec, unnorm_log_post_fun(theta_vec))
plt.xlabel(r"$\theta$");
plt.title(r"Unnormalized log-posterior  distribution of $\theta$");

The log-posterior has a much better scaling!

Actually, all we need to implement Metropolis is the ratio:
$$f_{\rm ratio}(\theta_{\rm proposal}, \theta_{\rm current}) = \frac{f_{\rm post}(\theta_{\rm proposal})}{f_{\rm post}(\theta_{\rm current})}$$
Then, $f_{\rm ratio}(\theta_{\rm proposal}, \theta_{\rm current})$  may be computed in a numerically stable way by exponentiating the difference between the log-posteriors at $\theta_{\rm proposal}$ and $\theta_{\rm current}$,  exploiting the identity:

$$f_{\rm ratio}(\theta_{\rm proposal}, \theta_{\rm current}) = \frac{f_{\rm post}(\theta_{\rm proposal})}{f_{\rm post}(\theta_{\rm current})} = e^{(\log f_{\rm post}(\theta_{\rm proposal})- \log f_{\rm post}(\theta_{\rm current}))}.$$

The additive constant disappears from the difference between the log-likelihood in 2 points, and the quantity to be exponentiated is not too large!

In general, the *golden rule* is to perform as many operations as possible in the log domain and go back only when it is strictly necessary.

* Implement a Python function `f_ratio_fun` to compute $f_{\rm ratio}(\theta_{\rm proposal}, \theta_{\rm current})$, exploiting the above trick

In [ ]:
def f_ratio_fun(theta_propose, theta_current):
       
    log_f_propose = unnorm_log_post_fun(theta_propose)
    log_f_current = unnorm_log_post_fun(theta_current)
    log_f_ratio = log_f_propose - log_f_current # log(p_prop/p_prev) = log(p_prop) - log(p_prev)
    f_ratio = np.exp(log_f_ratio)
    #f_ratio = np.nan_to_num(p_ratio, nan=0.0)
    return f_ratio

* Implement a Metropolis algorithm using the newly defined `f_ratio_fun`. Comment the results.

In [ ]:
theta_step = theta_0
thetas = [] # here we collect the theta values from the chain

for idx in range(N):
    thetas.append(theta_step)
    
    # Generate proposal
    theta_prop = theta_step + sigma*np.random.randn()
    theta_prop = np.clip(theta_prop, 0, 1)
    
    f_ratio = f_ratio_fun(theta_prop, theta_step) 
    accept_prob = np.minimum(1.0, f_ratio)
    
    accept = (np.random.rand() < accept_prob)
    
    if accept:
        theta_step = theta_prop

In [ ]:
plt.hist(thetas, density=True, bins=50, label="Metropolis samples")
plt.plot(theta_vec, post_var.pdf(theta_vec), label="Exact posterior")
plt.legend()
plt.xlim([0, 1]);

We obtain the same result as before.

* Try our different values of sigma and comment the results

In [ ]:
sigma = 0.1 # proposal standard deviation
theta_0 = 0.5 # a lucky guess...
N = 50_000 # chain length

theta_step = theta_0
thetas_test = [] # here we collect the theta values from the chain

for idx in range(N):
    thetas.append(theta_step)
    theta_prop = theta_step + sigma*np.random.randn()
    theta_prop = np.clip(theta_prop, 0, 1)
    
    f_ratio = f_ratio_fun(theta_prop, theta_step)
    accept_prob = np.minimum(1.0, p_ratio)
    accept = (np.random.rand() < accept_prob)
    
    if accept:
        theta_step = theta_prop

In [ ]:
#plt.hist(thetas, bins=50);

## Approach 4: Probabilistic Programming

* Implement a sample-based estimation of the posterior of $\theta$ with pymc3

The Beta-Binomial is the Hello World of probabilistic programming! No need to worry about implementation subtleties of sampling, pymc3 takes care of it!

In [ ]:
y

In [ ]:
# Approach 4: use pymc(3)

with pm.Model() as beta_binomial:
    theta = pm.Beta("theta", alpha=a, beta=b)
    y_obs = pm.Binomial("y_obs", n=n, p=theta, observed=y)
    trace_bb = pm.sample(10_000, random_seed=123, return_inferencedata=True)


In [ ]:
with beta_binomial:
    display(az.summary(trace_bb))

In [ ]:
np.mean(thetas), np.std(thetas) # comparison with hand-made metropolis

In [ ]:
az.plot_trace(trace_bb);

In [ ]:
az.plot_posterior(trace_bb, color="b");

In [ ]:
az.plot_posterior(trace_bb, color="C0")
plt.plot(theta_vec, post_var.pdf(theta_vec), color="C1")
post_L, post_H = post_var.interval(1-alpha)
plt.axvspan(xmin=post_L, xmax=post_H, alpha=0.1, color="C1", label=r"HDI 95%");